<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7eb0daafc89ed7ec26b1f50c26eabdca976b652ed2539e900a5824d7d878137e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-08 10:03:47
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.58 C
-------------------
Today PnL: -39.86 K (-0.25%)
Current PnL: -25.18 L (-16.06%)
CY Booked + Current PnL: -10.89 L (-6.95%)
-------------------
Total profit:  2.17 L
Total loss:  -27.36 L
-------------------
Total Booked + Current PnL: 16.09 L (12.39%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.05%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 94.20 L (59.64%)
Deployed:  1.30 C
Current:  1.58 C
CAGR/XIRR %: 10.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.68,15.12,5.45,21.40,16653.0,40131.0,305557.0,6.83,63.0,X-LC,5.53,14.0,2.41,1.93,22.40,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.39,5.73,12.76,19.23,20299.0,8622.0,159082.0,24.27,81.0,L-LC,6.64,182.0,0.42,1.01,28.22,XY25,ATH,MINING
80,TTKPRESTIG,770.00,0.20,-20.45,25.82,0.09,20700.0,-20605.0,80172.0,86.13,39.0,M-SC,3.20,253.0,-1.00,0.51,5.33,OX40N,NTT,DURABLES
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.15,-13.07,15.06,0.02,22852.0,-22820.0,151739.0,131.58,58.0,M-SC,7.80,240.0,-1.00,0.96,31.99,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TRIDENT,48.00,6.71,-25.03,71.43,28.51,49401.0,-23094.0,69160.0,-11.17,60.0,M-SC,7.02,226.0,-0.47,0.44,18.34,XR,NTT,TEXTILES
4,ALKYLAMINE,4546.37,6.61,-30.16,177.23,93.63,124976.0,-30447.0,70516.0,-25.23,53.0,H-SC,16.48,143.0,-0.24,0.45,6.60,SR,ATH,CHEMICALS
9,BAJAJHLDNG,14451.52,3.81,3.94,24.31,29.22,48043.0,7500.0,197625.0,1.33,58.0,X-LC,11.65,29.0,0.16,1.25,13.49,X40,ATH,FINANCE
71,STARHEALTH,761.00,2.97,-11.42,66.25,47.26,164973.0,-32107.0,249016.0,20.96,47.0,H-SC,7.76,174.0,-0.19,1.58,33.42,XY24,NTT,INSURANCE
46,JSWINFRA,342.00,1.18,-5.12,22.71,16.43,47786.0,-11356.0,210418.0,-19.71,49.0,X-MC,6.76,66.0,-0.24,1.33,24.18,X40N,NTT,REALTY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,-2.25,2.68,44.75,48.63,91878.0,5360.0,205314.0,-31.74,65.0,H-SC,13.55,137.0,0.06,1.30,14.61,XR,NTT,MISC
76,TCS,4389.97,-2.20,-11.38,36.21,20.71,121373.0,-43045.0,335192.0,-23.62,49.0,X-LC,2.61,3.0,-0.35,2.12,12.00,X40,ATH,IT
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE
75,TATAELXSI,9161.00,-1.56,-16.97,59.00,32.03,64586.0,-22367.0,109468.0,-10.16,68.0,H-SC,6.94,157.0,-0.35,0.69,22.61,OX40N,NTT,IT
87,WIPRO,420.00,-1.22,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,57.0,M-LC,6.24,101.0,0.16,1.18,16.49,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.22,-0.66,109.30,107.91,160304.0,-980.0,146664.0,-19.34,57.0,M-SC,10.17,220.0,-0.01,0.93,9.36,OX40N,ATH,CABLES
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.22,-0.66,109.30,107.91,160304.0,-980.0,146664.0,-19.34,57.0,M-SC,10.17,220.0,-0.01,0.93,9.36,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.15,-13.07,15.06,0.02,22852.0,-22820.0,151739.0,131.58,58.0,M-SC,7.80,240.0,-1.00,0.96,31.99,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,-1.56,-16.97,59.00,32.03,64586.0,-22367.0,109468.0,-10.16,68.0,H-SC,6.94,157.0,-0.35,0.69,22.61,OX40N,NTT,IT
80,TTKPRESTIG,770.00,0.20,-20.45,25.82,0.09,20700.0,-20605.0,80172.0,86.13,39.0,M-SC,3.20,253.0,-1.00,0.51,5.33,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.16,-20.49,42.72,13.47,91595.0,-55260.0,214407.0,-67.34,55.0,H-SC,4.72,159.0,-0.60,1.36,9.02,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.51,3.76,16.36,20.74,34489.0,7645.0,210815.0,35.83,72.0,M-MC,5.77,189.0,0.22,1.33,14.78,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.39,5.73,12.76,19.23,20299.0,8622.0,159082.0,24.27,81.0,L-LC,6.64,182.0,0.42,1.01,28.22,XY25,ATH,MINING
87,WIPRO,420.00,-1.22,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,57.0,M-LC,6.24,101.0,0.16,1.18,16.49,XR,NTT,IT
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE
37,IEX,219.00,-2.25,2.68,44.75,48.63,91878.0,5360.0,205314.0,-31.74,65.0,H-SC,13.55,137.0,0.06,1.30,14.61,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-1.22,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,57.0,M-LC,6.24,101.0,0.16,1.18,16.49,XR,NTT,IT
37,IEX,219.00,-2.25,2.68,44.75,48.63,91878.0,5360.0,205314.0,-31.74,65.0,H-SC,13.55,137.0,0.06,1.30,14.61,XR,NTT,MISC
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE
25,FINCABLES,1641.55,0.22,-0.66,109.30,107.91,160304.0,-980.0,146664.0,-19.34,57.0,M-SC,10.17,220.0,-0.01,0.93,9.36,OX40N,ATH,CABLES
1,ABB,7934.00,0.47,-2.32,49.02,45.57,125272.0,-6067.0,255552.0,-37.19,63.0,H-MC,4.56,121.0,-0.05,1.62,8.74,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.21,-16.53,32.73,10.78,65651.0,-39734.0,200584.0,-50.14,13.0,X-LC,5.77,1.0,-0.61,1.27,0.00,X40,NTT,FMCG
7,AWL,485.00,0.58,-25.34,110.53,57.19,269142.0,-82626.0,243501.0,-64.65,31.0,X-MC,8.76,58.0,-0.31,1.54,0.58,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.66,-11.05,46.83,30.62,98750.0,-26184.0,210870.0,-36.72,31.0,X-MC,16.14,57.0,-0.27,1.34,0.00,X40,ATH,APPARELS
22,DIXON,18931.72,0.51,-17.98,60.03,31.26,113625.0,-41488.0,189280.0,-61.69,31.0,X-MC,18.49,56.0,-0.37,1.20,0.89,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.71,-40.05,125.55,35.20,96839.0,-51538.0,77132.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,-0.60,0.60,20.04,20.77,47205.0,1410.0,235554.0,-8.21,37.0,X-MC,1.01,70.0,0.03,1.49,7.56,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,0.10,-15.73,48.94,25.51,76753.0,-29265.0,156830.0,-3.72,51.0,H-LC,1.51,50.0,-0.38,0.99,15.57,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.02,1.86,41.07,43.70,103778.0,4617.0,252685.0,-6.50,66.0,X-MC,2.10,73.0,0.04,1.60,17.19,XY24,BTT,FMCG
84,VBL,671.64,-0.64,2.30,32.62,35.67,105400.0,7273.0,323115.0,-9.83,68.0,X-LC,2.22,5.0,0.07,2.05,16.48,X40N,ATH,FMCG
45,JIOFIN,387.00,-0.53,-2.89,28.19,24.48,73191.0,-7731.0,259634.0,-4.49,54.0,X-LC,2.24,37.0,-0.11,1.64,50.49,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.71,-40.05,125.55,35.20,96839.0,-51538.0,77132.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
43,ITC,452.00,-0.21,-16.53,32.73,10.78,65651.0,-39734.0,200584.0,-50.14,13.0,X-LC,5.77,1.0,-0.61,1.27,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,-0.66,-11.05,46.83,30.62,98750.0,-26184.0,210870.0,-36.72,31.0,X-MC,16.14,57.0,-0.27,1.34,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,-0.90,-21.86,81.09,41.51,166883.0,-57565.0,205800.0,-6.97,35.0,X-MC,8.94,65.0,-0.34,1.30,0.22,XY25,ATH,FMCG
7,AWL,485.00,0.58,-25.34,110.53,57.19,269142.0,-82626.0,243501.0,-64.65,31.0,X-MC,8.76,58.0,-0.31,1.54,0.58,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.81,-26.14,93.39,42.84,44842.0,-16990.0,48016.0,-51.52,57.0,X-SC,27.83,83.0,-0.38,0.30,8.63,XY24,NTT,MISC
58,RELAXO,1176.00,-0.01,-47.96,192.65,52.31,145543.0,-69612.0,75548.0,-44.63,43.0,X-SC,7.71,92.0,-0.48,0.48,1.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.71,-40.05,125.55,35.20,96839.0,-51538.0,77132.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.21,2.16,21.45,24.08,27819.0,2741.0,129691.0,-6.13,56.0,X-SC,6.39,91.0,0.10,0.82,22.20,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.35,-15.96,70.54,43.32,96555.0,-25992.0,136880.0,-27.81,53.0,X-SC,7.76,90.0,-0.27,0.87,5.10,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.21,-16.53,32.73,10.78,65651.0,-39734.0,200584.0,-50.14,13.0,X-LC,5.77,1.0,-0.61,1.27,0.00,X40,NTT,FMCG
78,TMPV,600.00,-0.01,-25.54,65.15,22.98,109588.0,-57690.0,168208.0,-23.08,51.0,X-LC,4.45,2.0,-0.53,1.06,5.79,XY24,NTT,AUTO
76,TCS,4389.97,-2.20,-11.38,36.21,20.71,121373.0,-43045.0,335192.0,-23.62,49.0,X-LC,2.61,3.0,-0.35,2.12,12.00,X40,ATH,IT
84,VBL,671.64,-0.64,2.30,32.62,35.67,105400.0,7273.0,323115.0,-9.83,68.0,X-LC,2.22,5.0,0.07,2.05,16.48,X40N,ATH,FMCG
41,INFY,1972.00,-0.64,10.86,21.09,34.24,73155.0,33982.0,346870.0,-13.50,54.0,X-LC,7.67,6.0,0.46,2.20,20.04,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.21,-33.58,100.07,32.89,51319.0,-25924.0,51283.0,-733.49,65.0,L-MC,7.73,259.0,-0.51,0.32,41.23,XR,NTT,BANKS
49,MASFIN,398.61,-2.04,0.52,21.42,22.05,21097.0,510.0,98490.0,-13.28,56.0,H-SC,6.71,168.0,0.02,0.62,43.16,XR,ATH,FINANCE
13,BSOFT,831.70,-0.68,-20.16,88.85,50.79,99780.0,-28351.0,112302.0,2.44,57.0,H-SC,10.82,171.0,-0.28,0.71,30.76,XR,ATH,IT
72,SURYODAY,216.00,-0.26,-18.07,50.17,23.03,73608.0,-32354.0,146717.0,60.09,57.0,H-SC,5.06,165.0,-0.44,0.93,45.57,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.15,-13.07,15.06,0.02,22852.0,-22820.0,151739.0,131.58,58.0,M-SC,7.80,240.0,-1.00,0.96,31.99,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.45,-26.41,53.43,12.90,85907.0,-57712.0,160784.0,136.78,71.0,H-SC,10.45,173.0,-0.67,1.02,33.69,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,-1.56,-16.97,59.00,32.03,64586.0,-22367.0,109468.0,-10.16,68.0,H-SC,6.94,157.0,-0.35,0.69,22.61,OX40N,NTT,IT
17,COALINDIA,484.83,-0.39,5.73,12.76,19.23,20299.0,8622.0,159082.0,24.27,81.0,L-LC,6.64,182.0,0.42,1.01,28.22,XY25,ATH,MINING
85,VOLTAS,1856.00,-0.83,16.96,24.14,45.20,54138.0,32523.0,224265.0,5.55,69.0,X-MC,4.67,80.0,0.60,1.42,24.91,XY25,NTT,AC
57,RECLTD,446.00,-0.51,3.76,16.36,20.74,34489.0,7645.0,210815.0,35.83,72.0,M-MC,5.77,189.0,0.22,1.33,14.78,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.69
1,25,44.72
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.29
LC    32.64
MC    28.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     24.16
X40      21.68
XY25     18.28
X40N     11.71
XR        8.33
AR        7.94
OX40N     7.00
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    29.46
X-MC    22.12
H-SC    21.07
M-SC    10.34
X-SC     7.16
H-MC     4.30
M-MC     1.33
M-LC     1.18
L-LC     1.01
H-LC     0.99
L-SC     0.72
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.49,-7.63,42.94
IT,11.68,-15.97,74.46
FINANCE,10.23,-10.24,56.52
REFINERIES,9.76,-4.45,30.87
MISC,6.56,-25.18,76.39
ELECTRICAL,5.46,-8.97,48.88
PAINTS,5.22,-12.11,28.76
INSURANCE,4.29,-0.07,34.32
PHARMA,3.62,-2.69,35.01


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3152250.0,21
AR,1318185.0,10
XR,1174802.0,12
X40,1096400.0,15
X40N,894464.0,9
XY25,818807.0,10
OX40N,686221.0,10
SR,278945.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3467866.0,24
X-MC,1539557.0,16
M-SC,1424405.0,15
X-LC,1335041.0,16
X-SC,787791.0,8
H-MC,400422.0,3
L-SC,176144.0,2
M-LC,105988.0,1
H-LC,76753.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226859.0      6
           AR         909652.0      5
M-SC       XY24       821766.0      6
H-SC       XR         754811.0      7
X-LC       XY25       542998.0      6
X-MC       X40        494430.0      7
           XY24       435852.0      3
X-LC       X40        410647.0      6
X-MC       X40N       388254.0      4
X-SC       X40N       307593.0      3
M-SC       OX40N      297727.0      5
H-SC       OX40N      297599.0      4
X-SC       XY24       288875.0      3
H-SC       SR         278945.0      2
X-MC       XY25       221021.0      2
H-MC       AR         204303.0      2
X-LC       X40N       198617.0      2
H-MC       XY24       196119.0      1
X-SC       X40        191323.0      2
X-LC       XY24       182779.0      2
M-SC       XR         177435.0      2
           AR         127477.0      2
M-LC       XR         105988.0      1
L-SC       OX40N       90895.0      1
           XR          85249.0      1
H-LC       AR          76753.0      1
L-MC       XR          51319.0      1
M-MC       XY25        34489.0      1
L-LC       XY25        20299.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
